In [ ]:
# Finetuning LLM with HF

In [ ]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

checkpoint="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
input = ["It was a great way to waste your life", "How can someone be so nice"]

batch = tokenizer(input, padding = True, truncation =True, return_tensors = 'pt')

# adding labels to input
batch["labels"] = torch.tensor([0,1])

# settuping up adamw
optim = AdamW(model.parameters())

# taking 1 training step
optim.zero_grad()
loss = model(**batch).loss
print(loss.item())
loss.backward()
optim.step()
loss = model(**batch).loss
print(loss.item())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.7524076700210571
0.608144998550415


In [2]:
# let use fine-tune using a dataset

!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue","mrpc")      #MRPC (Microsoft Research Paraphrase Corpus) dataset used in GLUE benchmark
raw_datasets

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
raw_train = raw_datasets["train"]
raw_train      # dataset class

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

In [ ]:
raw_train[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [ ]:
# label: 1 or 0 , but which is what?
raw_train.features    # .features on dataset class not DataSetDict

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [ ]:
# label: type-> ClassLabel 0: not_eq  1:eq

In [ ]:
raw_train["sentence1"][:5]

['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .',
 'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .',
 'The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .']

In [ ]:
# Dataset class:  i) indexing with number to get dictionary (inp, lables, idx)  ii) features for metadata    iii) indexing with keys to get all values wrt that key in the dataset

In [ ]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# this is a bert model so we have token_type_ids (whether sen 1 or 2)
print(tokenizer.decode(inputs['input_ids']))
print(tokenizer.tokenize("This is the first sentences.","This is the second one."))
print(inputs['token_type_ids'])

[CLS] this is the first sentence. [SEP] this is the second one. [SEP]
['this', 'is', 'the', 'first', 'sentences', '.', 'this', 'is', 'the', 'second', 'one', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)
# problem with this is, it returns a dictionary (with our keys, input_ids, attention_mask, and token_type_ids, and values that are lists of lists)
# It will also only work if you have enough RAM to store your whole dataset during the tokenization
# padding with max-length of the largest statement in the dataset

In [ ]:
# let's use dataset.map() function of HF datasets

def tokenize_func(inp):
  return tokenizer(inp["sentence1"],inp["sentence2"],truncation = True)            # padding not set true intentionally

# it also works if the example dictionary contains several samples (each key as a list of sentences) since the tokenizer works on lists of pairs of sentences
# due to this we can set, batched=True in our call to map(), that will speed up computation

tokenized_datasets = raw_datasets.map(tokenize_func, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [ ]:
# we can pass num_procs to map() method and do multiprocessing but our tokenizer() itself uses multiple threads
# this becomes useful for custom tokenizer

In [ ]:
# Dynamic Padding
# Instead of padding initially to largest size, we can pad based on batches
# Dynamic Padding is usually implemented in DataLoader,
# The function that is responsible for putting together samples inside a batch is called a collate function.

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
x = tokenized_datasets["train"][1]    # sentence1, sentence2,idx,label,input_ids,token_type_ids, attention_mask
print(x)

{'sentence1': "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .", 'sentence2': "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .", 'label': 0, 'idx': 1, 'input_ids': [101, 9805, 3540, 11514, 2050, 3079, 11282, 2243, 1005, 1055, 2077, 4855, 1996, 4677, 2000, 3647, 4576, 1999, 2687, 2005, 1002, 1016, 1012, 1019, 4551, 1012, 102, 9805, 3540, 11514, 2050, 3079, 11282, 2243, 1005, 1055, 2077, 4855, 1996, 4677, 2000, 3647, 4576, 1999, 2687, 2005, 1002, 1016, 1012, 1019, 4551, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
samples = tokenized_datasets["train"][:4]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}    # label, mask, input_ids, token_type_ids(bert)

In [ ]:
[len(x) for x in samples["input_ids"]]       # different size of different tokenized sentences

[49, 53, 45, 81]

In [ ]:
# by dynamic padding all 4 should be padded to max-length (=81)
batch = data_collator(samples)

In [ ]:
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([4, 81]),
 'token_type_ids': torch.Size([4, 81]),
 'attention_mask': torch.Size([4, 81]),
 'labels': torch.Size([4])}

In [ ]:
 # hurray!

In [1]:
# Finetuning with Trainer API

from transformers import TrainingArguments, Trainer

# to pass the various arguments for training
training_args = TrainingArguments("test-trainer")         # only passed the save location

In [3]:
# performing model load, dataset preprocess, all at once again
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from transformers import DataCollatorWithPadding

checkpoint="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

raw_datasets = load_dataset("glue","mrpc")      #MRPC (Microsoft Research Paraphrase Corpus) dataset used in GLUE benchmark

def tokenize_func(inp):
  return tokenizer(inp["sentence1"],inp["sentence2"],truncation = True)

tokenized_datasets = raw_datasets.map(tokenize_func, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [4]:
# Warning message: Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

# BERT(our model) has not been pretrained on classifying pairs of sentences, so the head of the pretrained model(used of MLM and NSP) has been discarded and a new head suitable
# for sequence classification has been added instead.
# The warnings indicate that some weights were not used (the ones corresponding to the dropped pretraining head) and that some others were
# randomly initialized (the ones for the new head).
# This suggests us to fine tune the model

In [5]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,        # not mandatory to pass as it is the default only
    tokenizer=tokenizer,
)

<ipython-input-5-dba607424a42>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [6]:
# to start training (default metrics and device)
# trainer.train()

In [7]:
# evaluation_strategy: "steps" (evaluate every eval_steps) or "epoch" (evaluate at the end of each epoch).
# compute_metrics() function    to compute model performance while training instead of just loss function

In [4]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00


In [5]:
# for metrics we can use predefined metrics for datasets
import evaluate                    # of HF

metric = evaluate.load("glue", "mrpc")

In [6]:
import numpy as np

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)        # based on dim=1
    return metric.compute(predictions=predictions, references=labels)   # gives accuracy and f1 for mrpc

In [7]:
# adding eval_strategy and compute_metrics

training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-ac6513fe72f5>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
# switch to gpu
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ayushsur26 (ayushsur26-national-institute-of-technology-kurukshetra) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.418709,0.808824,0.875796
2,0.516300,0.614959,0.845588,0.895175
3,0.274300,0.718896,0.852941,0.896194


TrainOutput(global_step=1377, training_loss=0.3218682094510258, metrics={'train_runtime': 425.4846, 'train_samples_per_second': 25.862, 'train_steps_per_second': 3.236, 'total_flos': 405114969714960.0, 'train_loss': 0.3218682094510258, 'epoch': 3.0})

In [9]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(1725, 2) (1725,)


In [10]:
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8359420289855073, 'f1': 0.8809423643247791}

In [11]:
# Hurray!!! Training complete!

In [1]:
# Using Trainer API is too much abstraction....using Pytorch training loop instead next

In [1]:
!pip install datasets

In [2]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from transformers import DataCollatorWithPadding

checkpoint="bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

raw_datasets = load_dataset("glue","mrpc")      #MRPC (Microsoft Research Paraphrase Corpus) dataset used in GLUE benchmark

def tokenize_func(inp):
  return tokenizer(inp["sentence1"],inp["sentence2"],truncation = True)

tokenized_datasets = raw_datasets.map(tokenize_func, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Training preparation...
# we have to make a few changes on the dataset before making batches from it and loading it into the model
# we need to check what are the attributes and fields our model expects as input
# we have to set output format to pytorch tensors instead of lists

tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])     # unnecessary(not accepted by model)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")                      # label -> labels
tokenized_datasets.set_format("torch")       # setting up output format to pytorch tensor
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [4]:
# setting up pytorch dataloader for custom loop
from torch.utils.data import DataLoader

# passing collator fir padding, batch_size,dataset and shuffle option
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [8]:
### for TPU's , do not use dynamic padding for best performance...
### set padding = 'max_length' for optimal performance

In [5]:
# we need an iterator to fetch a batch
sample_batch = next(iter(train_dataloader))
{k: v.shape for k,v in sample_batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 81]),
 'token_type_ids': torch.Size([8, 81]),
 'attention_mask': torch.Size([8, 81])}

In [6]:
output = model(**sample_batch)
print(output.loss, output.logits.shape)      # whenever labels are attached, it produces loss with logits

tensor(0.5685, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [7]:
len(train_dataloader)

459

In [8]:
# seeting up optim and scheduler
# we can import from torch too (recommended)
from transformers import AdamW, get_scheduler

num_epoch = 3

optimizer = AdamW(model.parameters(), lr=3e-5)
num_training_steps = num_epoch * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
# let's do it
# moving model to cuda
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

device

device(type='cuda')

In [10]:
from tqdm.auto import tqdm         # progress_bar

progress_bar = tqdm(range(num_training_steps))
loss_sum = 0
val_loss_sum=0
model.train()
for epoch in range(num_epoch):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)

        optimizer.zero_grad()

        loss = outputs.loss
        loss_sum += loss.item()
        loss.backward()

        optimizer.step()
        lr_scheduler.step()

        progress_bar.update(1)

    print("Training Loss: ",loss_sum/len(train_dataloader))

    # custom steps:: more control at lower level using pytorch instead of Trainer API
    model.eval()
    for batch in eval_dataloader:
      batch = {k:v.to(device) for k,v in batch.items()}         # moving tensors to gpu
      with torch.no_grad():
        outputs = model(**batch)
      loss_val = outputs.loss
      val_loss_sum += loss_val.item()

    print("Validation Loss: ",val_loss_sum/len(eval_dataloader))

    val_loss_sum=0
    loss_sum=0

  0%|          | 0/1377 [00:00<?, ?it/s]

Training Loss:  0.5204700950800983
Validation Loss:  0.4571042416945976
Training Loss:  0.2424936873233656
Validation Loss:  0.3048662151045659
Training Loss:  0.07838777677827638
Validation Loss:  0.4092097062818414


In [11]:
! pip install evaluate

In [ ]:
## important error to keep in mind
## HF data collator is adding padding tokens to full_fill the batch size
# so for last batch when we have 24 samples, 8 extra are added by the collator...this cause a size mismatch in further operations

# Sol 1:
# data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=None)           # prevents adding extra batch
# SOl 2:
# slice logits in loop (as done below)

In [12]:
# using evaluation metrics like before
import evaluate

metric = evaluate.load('glue','mrpc')
model.eval()
for batch in eval_dataloader:
  batch = {k:v.to(device) for k,v in batch.items()}         # moving tensors to gpu
  with torch.no_grad():
    outputs = model(**batch)
  logits = output.logits[:batch['input_ids'].shape[0]]
  prediction = torch.argmax(logits,dim=-1)
  metric.add_batch(predictions=prediction, references=batch["labels"])         # to evaulate score for this batch

metric.compute()

{'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}

In [13]:
# slight fall in f1 due to overfitting but it is all-right...

In [ ]:
# to use multiple gpu's or cpu's for distributed training we use hf accelerate
# like torch.ddp but has more levels of abstraction like any other HF APIs...
# very easy to use with notebooks where DDP is difficult

# .py file
# commands
# accelerate config                      -- answer bunch of q to make a config file
# accelerate launch train.py             -- next command


In [15]:
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler
from accelerate import Accelerator
from accelerate import notebook_launcher            # for notebooks

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=2.5e-5)
accelerator = Accelerator()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

def train_func():
  progress_bar = tqdm(range(num_training_steps))

  model.train()
  for epoch in range(num_epochs):
      for batch in train_dataloader:
          # batch = {k: v.to(device) for k, v in batch.items()}
          outputs = model(**batch)
          loss = outputs.loss
          # loss.backward()
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)


notebook_launcher(train_func)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Launching training on one GPU.


  0%|          | 0/918 [00:00<?, ?it/s]

In [16]:
# using evaluation metrics like before
import evaluate

metric = evaluate.load('glue','mrpc')
model.eval()
for batch in eval_dataloader:
  batch = {k:v.to(device) for k,v in batch.items()}         # moving tensors to gpu
  with torch.no_grad():
    outputs = model(**batch)
  logits = output.logits[:batch['input_ids'].shape[0]]
  prediction = torch.argmax(logits,dim=-1)
  metric.add_batch(predictions=prediction, references=batch["labels"])         # to evaulate score for this batch

metric.compute()

{'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}